This code was adapted from:


*   https://www.geeksforgeeks.org/spelling-checker-in-python/
*   NLP Labs


In [ ]:
!apt install --upgrade libomp-dev

!pip install --upgrade transformers==3.0.2
!pip install --upgrade faiss-gpu==1.6.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!pip install python-terrier

In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git


  Cloning https://github.com/terrierteam/pyterrier_colbert.git to /tmp/pip-req-build-nbmvcnh6
  Running command git clone -q https://github.com/terrierteam/pyterrier_colbert.git /tmp/pip-req-build-nbmvcnh6
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to /tmp/pip-install-jt8krl44/colbert_a0308266f18a4e8698017ab96aa4c6e3
  Running command git clone -q https://github.com/cmacdonald/ColBERT.git /tmp/pip-install-jt8krl44/colbert_a0308266f18a4e8698017ab96aa4c6e3
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.


In [ ]:
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"


In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
#we need to import the following libraries.
import pandas as pd
#to display the full text on the notebook without truncation
#pd.set_option('display.max_colwidth', 150)
import numpy as np
import re
from snowballstemmer import stemmer
from tqdm import tqdm

In [ ]:
pip install pyspellchecker

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from pandas import DataFrame
from matplotlib import pyplot
import random
import nltk
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import rand
from sklearn.metrics.pairwise import cosine_similarity
from numpy import argsort
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker
nltk.download('punkt')
nltk.download('stopwords')

drive.mount("/content/drive")

data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 ArtsRatings_5000_test.txt	    JewelleryReviewsLSA.csv
 ArtsRatings_5000_train.txt	    JewelleryReviewsQueryRelevantID.csv
 ArtsReviews_5000_test.txt	    metamorphosis_clean.txt
 ArtsReviews_5000_train.txt	   'My CW1.ipynb'
'F135864 - Task 1.ipynb'	   'My CW 2.ipynb'
'F135864 - Task 2.ipynb'	   'MY CW 3.ipynb'
'F135864 - Task 3.ipynb'	    Reduced_ArtsRatings_5000.txt
'F135864 - Task 4.ipynb'	    Reduced_ArtsReviews_5000.txt
'F135864 - Task 5.ipynb'	    review_polarity
 glove.6B.100d.txt		    Untitled0.ipynb
 JewelleryReviewsDeepLearning.csv


In [ ]:
# load data
file = open(data_path + "JewelleryReviewsLSA.csv",'rt')
text = file.read()
file.close()
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
# convert to lower case
tokens = [w.lower() for w in tokens]
# stemming of words
porter = PorterStemmer()
tokens = [porter.stem(word) for word in tokens]

# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]

#remove columns' titles
words = words[3:]

print(words[:100])
print(len(words))

#Correct spelling of words
spell = SpellChecker()
x = spell.unknown(words)   

for i in range (len(words)):
    # Replace each word by its correct spelling
    words[i] = spell.correction(words[i])

#check that the length of the array did not change
print(len(words))  
print(words)

['expect', 'like', 'regular', 'size', 'ring', 'thi', 'one', 'look', 'like', 'ring', 'toy', 'someth', 'funi', 'mm', 'ring', 'thi', 'ring', 'may', 'cttw', 'round', 'diamond', 'solitair', 'ring', 'white', 'gold', 'order', 'ring', 'state', 'toe', 'ring', 'descript', 'ring', 'came', 'veri', 'quickli', 'wa', 'toe', 'ring', 'nt', 'know', 'anyon', 'size', 'toe', 'know', 'anyon', 'pleas', 'direct', 'thi', 'seller', 'wed', 'band', 'toe', 'ring', 'thumb', 'ring', 'pick', 'one', 'thi', 'ring', 'beauti', 'ring', 'first', 'shipment', 'thi', 'ring', 'scratch', 'diamond', 'wa', 'miss', 'diamond', 'shoulder', 'ring', 'return', 'ring', 'replac', 'ring', 'second', 'ring', 'wa', 'even', 'wors', 'clearli', 'visibl', 'white', 'scratch', 'right', 'middl', 'black', 'diamond', 'doe', 'look', 'great', 'thi', 'time', 'diamond', 'miss', 'shoulder', 'ring', 'return', 'refund']
3963
3963
['expect', 'like', 'regular', 'size', 'ring', 'thi', 'one', 'look', 'like', 'ring', 'toy', 'someth', 'fun', 'my', 'ring', 'thi', 

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/JewelleryReviewsLSA.csv')
data_all_reviews = df["Reviews"].to_numpy()


In [ ]:
queries

['d']

In [ ]:
# we need to import pandas. We commonly rename it to pd, to make commands shorter
import pandas as pd

# lets not truncate output too much
pd.set_option('display.max_colwidth', 150)

docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

docs_df

,docno,text
0,d1,this is the first document of many documents
1,d2,this is another document
2,d3,the topic of this document is unknown


In [ ]:
df['text'] = df['Reviews']
df['docno'] = df['ID'].astype(str)


In [ ]:
indexer = pt.DFIndexer("./index_3docs", overwrite=True)
index_ref = indexer.index(df["text"],df['docno'])
index_ref.toString()

'./index_3docs/data.properties'

In [ ]:
!ls -lh index_3docs/

total 120K
-rw-r--r-- 1 root root 2.7K Mar 18 10:24 data.direct.bf
-rw-r--r-- 1 root root 3.4K Mar 18 10:24 data.document.fsarrayfile
-rw-r--r-- 1 root root 3.0K Mar 18 10:24 data.inverted.bf
-rw-r--r-- 1 root root  73K Mar 18 10:24 data.lexicon.fsomapfile
-rw-r--r-- 1 root root  969 Mar 18 10:24 data.lexicon.fsomaphash
-rw-r--r-- 1 root root 3.4K Mar 18 10:24 data.lexicon.fsomapid
-rw-r--r-- 1 root root 3.9K Mar 18 10:24 data.meta-0.fsomapfile
-rw-r--r-- 1 root root 1.6K Mar 18 10:24 data.meta.idx
-rw-r--r-- 1 root root 4.1K Mar 18 10:24 data.meta.zdata
-rw-r--r-- 1 root root 4.1K Mar 18 10:24 data.properties


In [ ]:
index = pt.IndexFactory.of(index_ref)

#lets see what type index is.
type(index)

jnius.reflect.org.terrier.structures.Index

In [ ]:
print(index.getCollectionStatistics().toString())

Number of documents: 200
Number of terms: 868
Number of postings: 2756
Number of fields: 0
Number of tokens: 3362
Field names: []
Positions:   false



In [ ]:
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

00 (<class 'str'>) -> term656 Nt=1 TF=1 maxTF=1 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
09 (<class 'str'>) -> term373 Nt=1 TF=1 maxTF=1 @{0 1 6} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
1 (<class 'str'>) -> term37 Nt=8 TF=8 maxTF=1 @{0 3 2} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
10 (<class 'str'>) -> term246 Nt=3 TF=3 maxTF=1 @{0 10 2} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
10x (<class 'str'>) -> term634 Nt=1 TF=1 maxTF=1 @{0 14 6} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
16 (<class 'str'>) -> term623 Nt=1 TF=1 maxTF=1 @{0 16 4} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
16g (<class 'str'>) -> term213 Nt=1 TF=1 maxTF=1 @{0 18 2} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
17 (<class 'str'>) -> term488 Nt=1 TF=1 maxTF=1 @{0 19 4} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
18k (<class 'str'>) -> term5 Nt=1 TF=1 maxT

In [ ]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("document")

,docid,docno,rank,score,qid,query


Searching an index

In [38]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("ring")

,qid,docid,docno,rank,score,query
0,1,70,33620,0,10.0,ring
1,1,67,34529,1,9.0,ring
2,1,2,43515,2,8.0,ring
3,1,3,46347,3,8.0,ring
4,1,15,32496,4,8.0,ring
...,...,...,...,...,...,...
102,1,164,41319,102,1.0,ring
103,1,166,45146,103,1.0,ring
104,1,170,209,104,1.0,ring
105,1,171,28542,105,1.0,ring


In [39]:
import pandas as pd
queries = pd.DataFrame([["q1", "document"], ["q2", "beautiful"]], columns=["qid", "query"])
br.transform(queries)

,qid,docid,docno,rank,score,query
0,q2,158,45860,0,3.0,beautiful
1,q2,85,19548,1,2.0,beautiful
2,q2,101,10612,2,2.0,beautiful
3,q2,165,39932,3,2.0,beautiful
4,q2,168,8341,4,2.0,beautiful
5,q2,2,43515,5,1.0,beautiful
6,q2,3,46347,6,1.0,beautiful
7,q2,15,32496,7,1.0,beautiful
8,q2,16,35526,8,1.0,beautiful
9,q2,21,50650,9,1.0,beautiful


In [40]:
br(queries)

,qid,docid,docno,rank,score,query
0,q2,158,45860,0,3.0,beautiful
1,q2,85,19548,1,2.0,beautiful
2,q2,101,10612,2,2.0,beautiful
3,q2,165,39932,3,2.0,beautiful
4,q2,168,8341,4,2.0,beautiful
5,q2,2,43515,5,1.0,beautiful
6,q2,3,46347,6,1.0,beautiful
7,q2,15,32496,7,1.0,beautiful
8,q2,16,35526,8,1.0,beautiful
9,q2,21,50650,9,1.0,beautiful


In [ ]:
docs_df = pd.DataFrame([
        ["d1", "The ring is a great gift. My friend loves it"],
        ["d2", "horrible bad quality bracelet"],
        ["d3", "arrived promptly and happy with the seller"],
        ["d4", "wear it with casual wear"],
        ["d5", "i expected better quality. i will return this item"],
        ["d6", "looks beautiful. The design is pretty. pefect and color is light"],
        ["d7", "This ring looks nothing like the picture. the diamonds are small and not very noticeable"],
        ["d8", "braclet looked just like its picture and is nice quality sterling silver."],
    ], columns=["qid", "query"])

docs_df

,qid,query
0,d1,The ring is a great gift. My friend loves it
1,d2,horrible bad quality bracelet
2,d3,arrived promptly and happy with the seller
3,d4,wear it with casual wear
4,d5,i expected better quality. i will return this ...
5,d6,looks beautiful. The design is pretty. pefect ...
6,d7,This ring looks nothing like the picture. the ...
7,d8,braclet looked just like its picture and is ni...


In [ ]:
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
retr.setControl("wmodel", "TF_IDF")
retr.setControls({"wmodel": "TF_IDF"})
res=retr.transform(docs_df)
res 

,qid,docid,docno,rank,score,query
0,d1,114,41876,0,6.964222,The ring is a great gift. My friend loves it
1,d1,112,45278,1,5.543047,The ring is a great gift. My friend loves it
2,d1,110,56494,2,5.410054,The ring is a great gift. My friend loves it
3,d1,118,11247,3,5.304773,The ring is a great gift. My friend loves it
4,d1,55,34483,4,5.173286,The ring is a great gift. My friend loves it
...,...,...,...,...,...,...
646,d8,75,27793,128,0.797653,braclet looked just like its picture and is ni...
647,d8,2,43515,129,0.638809,braclet looked just like its picture and is ni...
648,d8,3,46347,130,0.638809,braclet looked just like its picture and is ni...
649,d8,15,32496,131,0.638809,braclet looked just like its picture and is ni...


In [ ]:
retr.search("Light")


,qid,docid,docno,rank,score,query
0,1,159,46500,0,4.073367,Light
1,1,152,44126,1,3.608071,Light
2,1,196,44490,2,3.502821,Light
3,1,186,37794,3,2.981065,Light
4,1,79,42604,4,2.908851,Light


In [ ]:
#QueryRelevantIDs
qrels = pd.DataFrame([
        ["1", "36164"],
        ["1", "58481"],
        ["1", "26246"],
        ["1", "2033"],
        ["2", "57123"],
        ["2", "25299"],
        ["2", "55017"],
        ["2", "7432"],
        ["3", "33251"],
        ["3", "17304"],
        ["3", "50019"],
        ["3", "27679"],
        ["4", "40373"],
        ["4", "28648"],
        ["4", "37486"],
        ["4", "30640"],
        
    ], columns=["qid", "docno"])

qrels

,qid,docno
0,1,36164
1,1,58481
2,1,26246
3,1,2033
4,2,57123
5,2,25299
6,2,55017
7,2,7432
8,3,33251
9,3,17304


In [ ]:
#Implementing additional functionality, taking input from user

queries = list()
f = True
while f:
  query = input('Pleas enter your query')
  queries.append(query)
  yn = input('Do you want to enter another queries? Please answer with Y or N')
  if yn=='N':
    f = False

Pleas enter your queryd
Do you want to enter another queries? Please answer with Y or NN


EVALUATION

In [ ]:
#eval = pt.Utils.evaluate(res,qrels)
#eval

#Code did not work, needed some changes